In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC, SVR
from sklearn.preprocessing import MinMaxScaler, minmax_scale
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import GridSearchCV, LeaveOneOut,KFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
import scipy.stats as stats
import pickle
import warnings
warnings.filterwarnings(action='ignore')

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score
from sklearn.metrics import precision_recall_fscore_support,roc_curve,auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
#from eli5.sklearn import PermutationImportance
#from imblearn.combine import SMOTETomek
from sklearn.impute import SimpleImputer
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from pandas import Series, DataFrame


from sklearn.metrics import precision_score, recall_score, f1_score
import random

In [2]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./HAR/UCI/features.txt',sep=' ',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./HAR/UCI/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./HAR/UCI/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./HAR/UCI/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./HAR/UCI/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 결정 트리에서 사용한 get_human_dataset( )을 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

print("## 학습 피처 데이터 정보 ##")
print(X_train.info())

## 학습 피처 데이터 정보 ##
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None


In [4]:
#스케일링
def imp(df):
    imputer = SimpleImputer(strategy='median')

    imputer.fit(df)
    output2 = imputer.transform(df)

    output2 = pd.DataFrame(output2, columns=df.columns, index=list(df.index.values))

    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(output2)
    output2 = min_max_scaler.transform(output2)
    output2 = pd.DataFrame(output2, columns=df.columns, index=list(df.index.values))

    datafrm = pd.DataFrame(output2)
    return datafrm

In [5]:
def df_maker(col_num, ind_num, fill):
    col = []
    ind = []
    con = []
    for i in range(0,col_num):
        col.append(fill)
    for i in range(0,ind_num):
        ind.append(fill)
    for i in range(0,ind_num):
        con.append(col)
    return pd.DataFrame(con, columns = col, index = ind)

In [6]:
LR = LogisticRegression()
RF = RandomForestClassifier()
DT = DecisionTreeClassifier()
SVM =svm.SVC(kernel = 'rbf',probability=True)

LR_grid={'penalty': ['none'], 
            'C': [0.001, 0.01, 0.1, 1, 10, 100]
                   }

RF_grid={'max_depth': [4,6,8,10,12,14], # max_depth: The maximum depth of the tree.
              'n_estimators': [100,200,400,600], # n_estimators: The number of trees in the forest.
              'min_samples_split': [50, 100,25,75,125]
              }

DT_grid={'min_samples_leaf': [1, 2, 4, 8, 16]} 

SVM_grid = {'gamma': [0.01, 0.1, 1, 10, 100], 
               'C': [0.01, 0.1, 1, 10, 100]}

gridlist = [LR_grid,RF_grid,DT_grid,SVM_grid]
classlist = [LR,RF,DT,SVM]


In [7]:
def gridsearch(gridlist,classlist,train_x,train_y):
    for i in range(4):
        grid = GridSearchCV(classlist[i], gridlist[i], scoring = 'roc_auc', cv=5,n_jobs=-1)  
        grid.fit(train_x, train_y)
        print(classlist[i])
        print(grid.best_params_)
        print(grid.best_score_)
        classlist[i] = grid.best_estimator_
    return classlist

In [8]:
classlist = gridsearch(gridlist,classlist,X_train,y_train)

LogisticRegression()
{'C': 0.001, 'penalty': 'none'}
nan
RandomForestClassifier()
{'max_depth': 4, 'min_samples_split': 50, 'n_estimators': 100}
nan
DecisionTreeClassifier()
{'min_samples_leaf': 1}
nan
SVC(probability=True)
{'C': 0.01, 'gamma': 0.01}
nan


In [9]:
LR = LogisticRegression(C= 0.001, penalty= 'none')
RF = RandomForestClassifier(max_depth= 4, min_samples_split= 50, n_estimators= 100)
XGB = DecisionTreeClassifier(min_samples_leaf = 1)
SVM =svm.SVC(kernel = 'rbf',C=0.01, gamma = 0.01,probability=True)

classlist = [LR,RF,XGB,SVM]



In [10]:
def cross_val(classlist,train_x,train_y):
    kf = StratifiedKFold(n_splits=5, shuffle=True)
    LR = classlist[0]
    RF = classlist[1]
    DT = classlist[2]
    SVM = classlist[3]
    
    LR.fit(train_x,train_y)
    RF.fit(train_x,train_y)
    DT.fit(train_x,train_y)
    SVM.fit(train_x,train_y)
    
    LR_scores = cross_val_score(LR, train_x, train_y, cv = kf)
    RF_scores = cross_val_score(RF, train_x, train_y, cv = kf)
    DT_scores = cross_val_score(DT, train_x, train_y, cv = kf)
    SVM_scores = cross_val_score(SVM, train_x, train_y, cv = kf)
        
    score_mean = [LR_scores.mean(), RF_scores.mean(), DT_scores.mean(), SVM_scores.mean()]
    
    score = df_maker(4,5,0)
    score.columns = ['LR_scores','RF_scores','DT_scores','SVM_scores']
    score[score.columns[0]] = LR_scores
    score[score.columns[1]] = RF_scores
    score[score.columns[2]] = DT_scores
    score[score.columns[3]] = SVM_scores
    score.index = [1,2,3,4,5]
    
    return score_mean

In [11]:
cross_val(classlist,X_train,y_train)

[0.9836780014521104,
 0.9197483316916161,
 0.9416497639164436,
 0.7251104112617174]

In [12]:
def CI(classlist,test_x,test_y):

    LR = classlist[0]
    RF = classlist[1]
    DT = classlist[2]
    SVM = classlist[3]
    # 테스트 데이터로 예측 수행
    LR_pred = LR.predict(test_x)
    RF_pred = RF.predict(test_x)
    DT_pred = DT.predict(test_x)
    SVM_pred = SVM.predict(test_x)

    # 평가 메트릭 계산
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    LR_accuracy = accuracy_score(test_y, LR_pred)
    RF_accuracy = accuracy_score(test_y, RF_pred)
    DT_accuracy = accuracy_score(test_y, DT_pred)
    SVM_accuracy = accuracy_score(test_y, SVM_pred)

    print("Logistic Regression 정확도:", LR_accuracy)
    print("Random Forest 정확도:", RF_accuracy)
    print("Decision Tree 정확도:", DT_accuracy)
    print("Support Vector Machine 정확도:", SVM_accuracy)

In [13]:
CI(classlist,X_test,y_test)

Logistic Regression 정확도: 0.9565659993213438
Random Forest 정확도: 0.8693586698337292
Decision Tree 정확도: 0.8581608415337632
Support Vector Machine 정확도: 0.7387173396674585


In [14]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()

# 모델 학습
lsvm.fit(X_train, y_train)

LinearSVC()

In [15]:
# 테스트 데이터로 예측 수행
y_pred = lsvm.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'정확도: {accuracy:.2f}')

정확도: 0.97
